In [2]:
import pandas as pd
import numpy as np
import json
import re

#https://bl.ocks.org/rofrischmann/0de01de85296591eb45a1dde2040c5a1

In [56]:
details = pd.read_csv("../Data/extra_details_complete.csv",index_col=0)

# Sort database by descending metascore
details = details.sort_values('metascore',ascending=False)

details.head(10)

,metascore,name,console,userscore,date,genre(s),developer,publisher,rating,esrb descriptors,number of online players,special controllers,number of players,official site,franchise
1301,98,Grand Theft Auto IV,X360,7.9,"Apr 29, 2008","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,Blood Intense Violence Partial Nudity Strong L...,16 Players Online Up to 16 Players,Headset,1 Player,http://www.rockstargames.com/IV/,Grand Theft Auto
200,98,Red Dead Redemption 2,XONE,5.9,"Oct 26, 2018","Action Adventure, Open-World",Rockstar Games,Rockstar Games,M,,Up to 32 Players,,,,Red Dead
1400,98,Tony Hawk's Pro Skater 2,PS,7.4,"Sep 20, 2000","Alternative, Skateboarding, Sports",Neversoft Entertainment,Activision,T,Mild Animated Violence Mild Language,,,1-2 Players,,Tony Hawk Skateboarding
1300,98,Grand Theft Auto IV,PS3,7.5,"Apr 29, 2008","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,Blood Intense Violence Partial Nudity Strong L...,16 Players Online Up to 16 Players,,1 Player,http://www.rockstargames.com/IV/,Grand Theft Auto
0,97,Grand Theft Auto V,PS3,8.3,"Sep 17, 2013","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,,Up to 16 Players,,,http://www.rockstargames.com/,Grand Theft Auto
201,97,Red Dead Redemption 2,PS4,7.7,"Oct 26, 2018","Action Adventure, Open-World",Rockstar Games,Rockstar Games,M,,Up to 32 Players,,,,Red Dead
2300,97,Metroid Prime,GC,9.2,"Nov 17, 2002","Action, First-Person, Sci-Fi, Shooter",Retro Studios,Nintendo,T,Violence,,,1 Player,,Metroid
321,97,The Legend of Zelda: Breath of the Wild,Switch,8.5,"Mar 3, 2017","Action Adventure, Open-World",Nintendo,Nintendo,E10+,,No Online Multiplayer,,,,Legend of Zelda
743,97,Super Mario Galaxy 2,WII,9.1,"May 23, 2010","3D, Action, Platformer",Nintendo EAD Tokyo,Nintendo,E,,No Online Multiplayer,,,http://www.nintendo.com/games/detail/nR48Ll934...,Mario
356,97,Super Mario Odyssey,Switch,8.9,"Oct 27, 2017","3D, Action, Platformer",Nintendo,Nintendo,E10+,,No Online Multiplayer,,,,Mario


## Find specific game and all its neigbours

In [57]:
# Filter database first for one platform only
ps3_details = details.loc[details['console']=='PS3']

# Finding GTA V
GTAV_ps3 = ps3_details.loc[ps3_details['name'] == 'Grand Theft Auto V']
GTAV_ps3

,metascore,name,console,userscore,date,genre(s),developer,publisher,rating,esrb descriptors,number of online players,special controllers,number of players,official site,franchise
0,97,Grand Theft Auto V,PS3,8.3,"Sep 17, 2013","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,,Up to 16 Players,,,http://www.rockstargames.com/,Grand Theft Auto


In [5]:
# Extracting all relevant info
GTAV_genres = GTAV_ps3['genre(s)'][0].split(',')
print(GTAV_genres)
GTAV_devs = GTAV_ps3['developer'][0].split(',')
print(GTAV_devs)
GTAV_pub = GTAV_ps3['publisher'][0].split(',')
print(GTAV_pub)
#and so forth

['Action Adventure', ' Modern', ' Open-World']
['Rockstar North']
['Rockstar Games']


In [58]:
# Filtering dataframe for matches to extracted info
games_in_same_genre = {}
for genre in GTAV_genres:
    # Regex matchs either a comma following or the end of line so Action won't match Action Adventure
    games_in_same_genre[genre] =  ps3_details.loc[ps3_details['genre(s)'].str.contains(genre+'(,|$)')]
    print(games_in_same_genre[genre]['name'],games_in_same_genre[genre]['genre(s)'])

games_by_same_dev = {}
for dev in GTAV_devs:
    games_by_same_dev[dev] =  ps3_details.loc[ps3_details['developer'].str.contains(dev+'(,|$)')]
    print(games_by_same_dev[dev]['name'])

games_by_same_pub = {}
for pub in GTAV_pub:
    games_by_same_pub[pub] =  ps3_details.loc[ps3_details['publisher']==pub]
    print(games_by_same_pub[pub]['name'])


1300                                   Grand Theft Auto IV
0                                       Grand Theft Auto V
238                                    Batman: Arkham City
878                             Uncharted 2: Among Thieves
2                                           The Last of Us
767                                    Red Dead Redemption
1305              Metal Gear Solid 4: Guns of the Patriots
271            The ICO & Shadow of the Colossus Collection
270                         Uncharted 3: Drake's Deception
773                                         God of War III
913                                    Assassin's Creed II
916                                  God of War Collection
919                                  Batman: Arkham Asylum
491                                               Okami HD
804                          Assassin's Creed: Brotherhood
544                                             Dishonored
337                         Metal Gear Solid HD Collecti

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  # Remove the CWD from sys.path while we load stuff.


## Need to make a list of dictionaries for the links 
e.g. 
[{'source': 'AAL', 'target': 'AAR', 'strength': 0.1},

 {'source': 'AAL', 'target': 'BLL', 'strength': 0.3},
 
 {'source': 'AAL', 'target': 'CPH', 'strength': 0.2},
 
 {'source': 'AAL', 'target': 'LGW', 'strength': 0.1},]

In [121]:
def MakeTopNLinks(key, values_df, N, already_included):
    topn_links = []
    i = 0
    names = list(values_df['name'])
    for name in names:
        #print(name)
        if i >= N:
            break
        if name in already_included:
            link_dict = {'source':key, 'target': name, 'strength': 0.001}
        else:
            link_dict = {'source':key, 'target': name, 'strength': 0.1}
            already_included.append(name)
            i += 1
        topn_links.append(link_dict)
    return topn_links, already_included

In [126]:
topn_by_pub, inc_pub = MakeTopNLinks('Rockstar Games', games_by_same_pub['Rockstar Games'], 10, ['Grand Theft Auto V'])
print(inc_pub)
topn_by_action_adv, inc_act_adv = MakeTopNLinks('Action Adventure', games_in_same_genre['Action Adventure'], 10, inc_pub)
print(inc_act_adv)
print(topn_by_pub + topn_by_action_adv)

['Grand Theft Auto V', 'Grand Theft Auto IV', 'Red Dead Redemption', 'L.A. Noire', 'Max Payne 3', 'Red Dead Redemption: Undead Nightmare Pack', 'Midnight Club: Los Angeles', 'Red Dead Redemption: Liars and Cheats']
['Grand Theft Auto V', 'Grand Theft Auto IV', 'Red Dead Redemption', 'L.A. Noire', 'Max Payne 3', 'Red Dead Redemption: Undead Nightmare Pack', 'Midnight Club: Los Angeles', 'Red Dead Redemption: Liars and Cheats', 'Batman: Arkham City', 'Uncharted 2: Among Thieves', 'The Last of Us', 'Metal Gear Solid 4: Guns of the Patriots', 'The ICO & Shadow of the Colossus Collection', "Uncharted 3: Drake's Deception", 'God of War III', "Assassin's Creed II", 'God of War Collection', 'Batman: Arkham Asylum']
[{'source': 'Rockstar Games', 'target': 'Grand Theft Auto IV', 'strength': 0.1}, {'source': 'Rockstar Games', 'target': 'Grand Theft Auto V', 'strength': 0.01}, {'source': 'Rockstar Games', 'target': 'Red Dead Redemption', 'strength': 0.1}, {'source': 'Rockstar Games', 'target': 'L.

## Need to also make a list of dictionaries for nodes

In [108]:
def MakeNodes():
    pass
GTAV_ps3

,metascore,name,console,userscore,date,genre(s),developer,publisher,rating,esrb descriptors,number of online players,special controllers,number of players,official site,franchise
0,97,Grand Theft Auto V,PS3,8.3,"Sep 17, 2013","Action Adventure, Modern, Open-World",Rockstar North,Rockstar Games,M,,Up to 16 Players,,,http://www.rockstargames.com/,Grand Theft Auto


In [118]:
# Making them manually first as a test

gtav_root = {'id':'Grand Theft Auto V', 'group':1, 'label':'Grand Theft Auto V\n Rating: M\n, Metascore: 97\n, User score: 8.3\n', 'level':1}
other_game_nodes = []
for name in inc_act_adv:
    if name == 'Grand Theft Auto V':
        continue
    other_game_nodes.append({'id':name, 'group':1,'label':name,'level':3})
act_adv_node = {'id':'Action Adventure', 'group':3, 'label':'Action Adventure', 'level':2}
pub_node = {'id':'Rockstar Games', 'group':2, 'label':'Rockstar Games','level':2}

[gtav_root,act_adv_node, pub_node]+other_game_nodes

[{'id': 'Grand Theft Auto V',
  'group': 1,
  'label': 'Grand Theft Auto V\n Rating: M\n, Metascore: 97\n, User score: 8.3\n',
  'level': 1},
 {'id': 'Action Adventure',
  'group': 3,
  'label': 'Action Adventure',
  'level': 2},
 {'id': 'Rockstar Games', 'group': 2, 'label': 'Rockstar Games', 'level': 2},
 {'id': 'Grand Theft Auto IV',
  'group': 1,
  'label': 'Grand Theft Auto IV',
  'level': 3},
 {'id': 'Red Dead Redemption',
  'group': 1,
  'label': 'Red Dead Redemption',
  'level': 3},
 {'id': 'L.A. Noire', 'group': 1, 'label': 'L.A. Noire', 'level': 3},
 {'id': 'Max Payne 3', 'group': 1, 'label': 'Max Payne 3', 'level': 3},
 {'id': 'Red Dead Redemption: Undead Nightmare Pack',
  'group': 1,
  'label': 'Red Dead Redemption: Undead Nightmare Pack',
  'level': 3},
 {'id': 'Midnight Club: Los Angeles',
  'group': 1,
  'label': 'Midnight Club: Los Angeles',
  'level': 3},
 {'id': 'Red Dead Redemption: Liars and Cheats',
  'group': 1,
  'label': 'Red Dead Redemption: Liars and Cheats',